In [1]:
import os
from functools import partial

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
DATA_PATH = '/Volumes/Extreme SSD/facial_emotion_recognition/data'
LABELS_FNAME = 'label.csv'
TRAIN_PATH = os.path.join(DATA_PATH, 'FER2013Train')
VAL_PATH = os.path.join(DATA_PATH, 'FER2013Valid')
TEST_PATH = os.path.join(DATA_PATH, 'FER2013Test')

In [3]:
LABELS = [
    'neutral',
    'happiness',
    'surprise',
    'sadness',
    'anger',
    'disgust',
    'fear',
    'contempt',
    'unknown',
    'not_a_face'
]
DF_COLUMNS = ['img_name', 'bbox', *LABELS]
N_CLASSES = len(LABELS)

## Data Analysis
Perform a small Exploratory Data Analysis on the train label CSVs, in order to check the labels distribution and bboxes.

In [4]:
df = pd.read_csv(
    os.path.join(TRAIN_PATH, LABELS_FNAME),
    header=None, index_col=None, names=DF_COLUMNS)
df.head()

,img_name,bbox,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,not_a_face
0,fer0000000.png,"(0, 0, 48, 48)",4,0,0,1,3,2,0,0,0,0
1,fer0000001.png,"(0, 0, 48, 48)",6,0,1,1,0,0,0,0,2,0
2,fer0000002.png,"(0, 0, 48, 48)",5,0,0,3,1,0,0,0,1,0
3,fer0000003.png,"(0, 0, 48, 48)",4,0,0,4,1,0,0,0,1,0
4,fer0000004.png,"(0, 0, 48, 48)",9,0,0,1,0,0,0,0,0,0


In [5]:
# expand the image names to the full path
df['img_name'] = TRAIN_PATH + os.sep + df['img_name']

In [6]:
# generate the label based on the annotations
df['label'] = np.argmax(df.iloc[:, 2:], axis=1)
df['label'].value_counts()

label
0    10295
1     7526
2     3557
3     3530
4     2463
6      655
5      191
8      171
7      168
9        2
Name: count, dtype: int64

In [7]:
df['bbox'].value_counts()

bbox
(0, 0, 48, 48)    28558
Name: count, dtype: int64

In [8]:
# remove redundant columns
# since all bboxes have the same value, they all can be removed
df = df.drop(LABELS + ['bbox'], axis=1)

In [9]:
df.head()

,img_name,label
0,/Volumes/Extreme SSD/facial_emotion_recognitio...,0
1,/Volumes/Extreme SSD/facial_emotion_recognitio...,0
2,/Volumes/Extreme SSD/facial_emotion_recognitio...,0
3,/Volumes/Extreme SSD/facial_emotion_recognitio...,0
4,/Volumes/Extreme SSD/facial_emotion_recognitio...,0


### Check Validation and Test Sets

In [10]:
val_df = pd.read_csv(
    os.path.join(VAL_PATH, LABELS_FNAME),
    header=None, index_col=None, names=DF_COLUMNS)
val_df.head()

,img_name,bbox,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,not_a_face
0,fer0028638.png,"(0, 0, 48, 48)",4,0,0,0,1,0,0,4,1,0
1,fer0028639.png,"(0, 0, 48, 48)",1,0,0,1,0,2,0,6,0,0
2,fer0028640.png,"(0, 0, 48, 48)",7,0,0,0,2,0,0,1,0,0
3,fer0028641.png,"(0, 0, 48, 48)",5,5,0,0,0,0,0,0,0,0
4,fer0028642.png,"(0, 0, 48, 48)",0,10,0,0,0,0,0,0,0,0


In [11]:
val_df['label'] = np.argmax(val_df.iloc[:, 2:], axis=1)
val_df['label'].value_counts()

label
0    1329
1     899
2     458
3     415
4     325
6      71
5      32
7      26
8      23
9       1
Name: count, dtype: int64

In [12]:
val_df['bbox'].value_counts()

bbox
(0, 0, 48, 48)    3579
Name: count, dtype: int64

In [13]:
test_df = pd.read_csv(
    os.path.join(TEST_PATH, LABELS_FNAME),
    header=None, index_col=None, names=DF_COLUMNS)
test_df.head()

,img_name,bbox,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,not_a_face
0,fer0032220.png,"(0, 0, 48, 48)",0,0,0,0,2,1,0,7,0,0
1,fer0032222.png,"(0, 0, 48, 48)",3,0,0,5,0,0,0,0,2,0
2,fer0032223.png,"(0, 0, 48, 48)",6,0,1,2,0,0,0,0,1,0
3,fer0032224.png,"(0, 0, 48, 48)",0,0,4,0,1,2,3,0,0,0
4,fer0032225.png,"(0, 0, 48, 48)",0,0,0,0,9,0,1,0,0,0


In [14]:
test_df['label'] = np.argmax(test_df.iloc[:, 2:], axis=1)
test_df['label'].value_counts()

label
0    1258
1     928
2     447
3     446
4     321
6      97
7      28
8      28
5      20
Name: count, dtype: int64

## Create TF Dataset

In [15]:
ds = tf.data.Dataset.from_tensor_slices((df['img_name'], df['label']))
next(iter(ds))

2024-03-25 12:45:46.855186: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-03-25 12:45:46.855221: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-03-25 12:45:46.855228: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-03-25 12:45:46.855244: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-25 12:45:46.855259: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


(<tf.Tensor: shape=(), dtype=string, numpy=b'/Volumes/Extreme SSD/facial_emotion_recognition/data/FER2013Train/fer0000000.png'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [16]:
ds = ds.shuffle(df.shape[0])
next(iter(ds))

(<tf.Tensor: shape=(), dtype=string, numpy=b'/Volumes/Extreme SSD/facial_emotion_recognition/data/FER2013Train/fer0020098.png'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [17]:
@tf.numpy_function(Tout=(tf.uint8, tf.int64))
def read_gs_image(img_path, label):
    return cv2.imread(img_path.decode("utf-8"), 0), label

ds = ds.map(read_gs_image, num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds))

(<tf.Tensor: shape=(48, 48), dtype=uint8, numpy=
 array([[182, 180, 176, ...,  50,  76, 136],
        [176, 173, 168, ...,  67,  84, 159],
        [172, 168, 161, ...,  69,  73, 144],
        ...,
        [ 35,  47, 105, ...,  11,  41,  34],
        [ 47,  64, 102, ...,  35,  49,  30],
        [ 65,  81, 123, ...,  57,  46,  36]], dtype=uint8)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [18]:
ds = ds.batch(32)
next(iter(ds))

(<tf.Tensor: shape=(32, 48, 48), dtype=uint8, numpy=
 array([[[ 60,  63,  60, ..., 201, 196, 182],
         [ 62,  63,  59, ..., 193, 194, 182],
         [ 62,  63,  58, ..., 169, 194, 191],
         ...,
         [ 52,  57,  63, ...,  58,  58,  61],
         [ 55,  59,  59, ...,  57,  59,  61],
         [ 61,  61,  58, ...,  62,  62,  62]],
 
        [[ 50,  43,  54, ...,  16,  16,  16],
         [ 56,  54,  68, ...,  17,  16,  16],
         [ 67,  75,  79, ...,  18,  17,  17],
         ...,
         [106, 103,  97, ...,  56,  48,  40],
         [103, 103, 102, ...,  63,  59,  49],
         [ 99, 103, 105, ...,  68,  70,  53]],
 
        [[158, 141,  62, ...,  42, 114, 106],
         [149, 100,  59, ...,  33,  69,  76],
         [120,  52,  73, ...,  19,  47,  74],
         ...,
         [217, 197, 174, ..., 137,  95,  48],
         [185, 182, 185, ..., 130, 141, 107],
         [200, 203, 178, ..., 125, 138, 191]],
 
        ...,
 
        [[ 37,  40,  23, ..., 134, 132,  90],
       

In [19]:
@tf.function
def normalize_img(img_batch, label_batch):
    # normalize image to the range of [-1, 1]
    return 2*(img_batch / 255) - 1, label_batch

ds = ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds))

(<tf.Tensor: shape=(32, 48, 48), dtype=float32, numpy=
 array([[[-0.9843137 , -0.9843137 , -0.9843137 , ..., -0.99215686,
          -0.9843137 , -0.99215686],
         [-0.9843137 , -0.9843137 , -0.9843137 , ..., -0.9843137 ,
          -0.9843137 , -0.99215686],
         [-0.96862745, -0.96862745, -0.9764706 , ..., -0.9843137 ,
          -0.9764706 , -0.9843137 ],
         ...,
         [-0.8509804 , -0.64705884, -0.67058825, ..., -0.75686276,
          -0.7490196 , -0.70980394],
         [-0.96862745, -0.7019608 , -0.654902  , ..., -0.7411765 ,
          -0.70980394, -0.7647059 ],
         [-1.        , -0.8117647 , -0.654902  , ..., -0.75686276,
          -0.8666667 , -0.8901961 ]],
 
        [[-0.8352941 , -0.79607844,  0.10588241, ...,  0.36470592,
          -0.2235294 , -0.1372549 ],
         [ 0.38823533,  0.09019613,  0.56078434, ...,  0.37254906,
          -0.2235294 , -0.1372549 ],
         [ 0.32549024,  0.32549024,  0.7254902 , ...,  0.28627455,
          -0.2235294 , -0.145

In [20]:
@tf.function
def label_to_one_hot(img_batch, label_batch, n_classes: int = 10):
    return img_batch, tf.one_hot(label_batch, n_classes)

ds = ds.map(
    partial(label_to_one_hot, n_classes=N_CLASSES),
    num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds))

(<tf.Tensor: shape=(32, 48, 48), dtype=float32, numpy=
 array([[[-0.5764706 , -0.7411765 , -0.8352941 , ...,  0.94509804,
           0.94509804,  0.8901961 ],
         [-0.6862745 , -0.85882354, -0.67058825, ...,  0.9372549 ,
           0.94509804,  0.88235295],
         [-0.56078434, -0.9137255 , -0.7019608 , ...,  0.9372549 ,
           0.94509804,  0.88235295],
         ...,
         [-0.21568626, -0.36470586, -0.31764704, ...,  0.9137255 ,
           0.92156863,  0.8666667 ],
         [-0.23921567, -0.38823527, -0.3490196 , ...,  0.9137255 ,
           0.92156863,  0.8666667 ],
         [-0.26274508, -0.41176468, -0.3960784 , ...,  0.9137255 ,
           0.9137255 ,  0.85882354]],
 
        [[ 0.45098042,  0.34901965,  0.28627455, ..., -0.12941176,
          -0.23921567, -0.3098039 ],
         [-0.11372548, -0.03529412, -0.1372549 , ..., -0.05882353,
          -0.17647058, -0.30196077],
         [-0.1607843 , -0.26274508, -0.38039213, ..., -0.02745098,
          -0.11372548, -0.262